In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import cross_val_score

### Cargo los archivos

* Previamente los genere con las notebook subidas
* Los avisos tienen 250 columnas con las matrices 'UxE' de la salida de aplicar SVD,
    pero por ahora no las uso.
* Las vistas contienen la cantidad de vistas

In [2]:
vist = pd.read_csv('/home/pcsor/facu/TP2/vistas.csv',low_memory=False)
post = pd.read_csv('/home/pcsor/facu/TP2/postulaciones.csv',low_memory=False)
postulantes = pd.read_csv('/home/pcsor/facu/TP2/postulantes.csv',low_memory=False)
avisos = pd.read_csv('/home/pcsor/facu/TP2/avisos5.csv',low_memory=False)

In [3]:
post.shape, avisos.shape

((6603752, 3), (25288, 254))

In [4]:
avisos.columns ,post.columns,vist.columns

(Index(['idaviso', 'tipo_de_trabajo', 'nombre_area', '0', '1', '2', '3', '4',
        '5', '6',
        ...
        '241', '242', '243', '244', '245', '246', '247', '248', '249',
        'nivel_laboral'],
       dtype='object', length=254),
 Index(['idaviso', 'idpostulante', 'fechapostulacion'], dtype='object'),
 Index(['idAviso', 'idpostulante', 'cantidadDeVistas'], dtype='object'))

In [5]:
###### Drop a las columnas que no se utilizan
features = list(set(avisos.columns) - set(['idaviso', 'nivel_laboral','tipo_de_trabajo','nombre_area']))
avisos = avisos.drop(features,axis=1)

In [8]:
vist.columns = ['idaviso','idpostulante','miro']
post['sepostulo'] = 1

In [9]:
##### Se arma un Trainset con dummys
train = pd.DataFrame()
train['idaviso'] = shuffle(post['idaviso'][0:3000000]).reset_index()['idaviso']
train['idpostulante'] = shuffle(post['idpostulante'][0:3000000]).reset_index()['idpostulante']
train['sepostulo'] = 0

In [10]:
##### Se controla que los dummys con '0' NO sean postlaciones reales
train = pd.merge(train, post, how='left', on=['idpostulante', 'idaviso'], indicator=True)
train = train.query('_merge != "both"')
train = train.drop(['_merge','sepostulo_y','fechapostulacion'],axis=1)
train.columns = ['idaviso','idpostulante','sepostulo']

In [11]:
train = train.drop_duplicates(['idaviso','idpostulante'])

In [12]:
train.shape

(2888040, 3)

In [13]:
###### Se filtran las postulaciones antiguas
post = post.sort_values(by='fechapostulacion',ascending=False)
post = post.reset_index().drop('index',axis=1)
abril = '2018-04-12 00:00:00'
post = post.loc[post['fechapostulacion']>abril]
post = post.drop(['fechapostulacion'], axis=1)
post[0:3]

In [21]:
####### Se juntan los dummys con las postulaciones filtradas
train = pd.concat([train,post])
train = shuffle(train)
train = train.reset_index().drop('index',axis=1)
train = train.drop_duplicates(['idaviso','idpostulante'])

In [25]:
####### Se agregan los features de los avisos
train = pd.merge(train,avisos,how='left',on='idaviso',indicator=True)
train = train.query('_merge != "left_only"')
train = train.drop(['_merge'],axis=1)
train[0:3]

In [41]:
####### Agrego la cantidad de vistas y relleno con '0' las que no son vistas
train = pd.merge(train,vist,how='left',on=['idpostulante','idaviso'])
train['miro_x'] = train['miro_x'].fillna(0)
train = train.drop(['miro_y'],axis=1)
train.columns = ['idaviso','idpostulante','sepostulo','tipo_de_trabajo','nombre_area','nivel_laboral','miro']

In [43]:
####### Agrego los datos de los postulantes y se rellenan los NaNs con el promedio
train = pd.merge(train,postulantes,how='left',on=['idpostulante'])
train['edad'] = train['edad'].fillna(int(train['edad'].mean()))
train['nombre'] = train['nombre'].fillna(int(train['nombre'].mean()))
train = train.drop('estado',axis = 1)

In [49]:
####### Se realiza un encoding de los IDS
lb_make = LabelEncoder()
train['idpostulante'] = lb_make.fit_transform(train['idpostulante'] )
train['idaviso'] = lb_make.fit_transform(train['idaviso'] )

### Cross validation con RF en modo regressor

* Se hacen 5 pruebas.
* Se usan 40 arboles para cada prueba.

In [51]:
features = list(set(train.columns) - set(['sepostulo']))
clf = RandomForestRegressor(n_estimators = 40, n_jobs = -1)
scores = cross_val_score(clf, train[features], train['sepostulo'], cv=5)

In [52]:
scores

array([0.9671955 , 0.9663511 , 0.96618453, 0.96828009, 0.96581121])